# 서울지하철데이터 pandas로 분석하기

- Tmoney 대중교통 통계자료 > 2022년 6월 교통카드 데이터 > 지하철 유무임별 이용현황
- 승차비율 50% 이하인 지하철역 

In [3]:
import pandas as pd

subway = pd.read_csv('./data/seoul_subway.csv', encoding='utf-8')
subway.head()

,사용월,호선명,역ID,지하철역,유임승차,유임하차,무임승차,무임하차
0,2022-06,1호선,150,서울역,1150754,1123193,194717,187155
1,2022-06,1호선,151,시청,537067,543569,80671,78279
2,2022-06,1호선,152,종각,834173,815704,131987,121717
3,2022-06,1호선,153,종로3가,450317,404106,299617,278151
4,2022-06,1호선,154,종로5가,394420,404105,254236,244555


In [4]:
# 빈 값이 있는지 확인
subway.isnull().sum()

사용월     0
호선명     0
역ID     0
지하철역    0
유임승차    0
유임하차    0
무임승차    0
무임하차    0
dtype: int64

In [5]:
# 데이터 수정 위해서 subway2 생성
subway2 = subway.copy()

In [6]:
# 전체탑승객, 유임승차비율, 무임승차비율 데이터 추가
subway2['전체탑승객'] = subway['유임승차'] + subway['무임승차']
subway2['유임승차비율'] = round((subway2['유임승차']  / subway2['전체탑승객']) * 100, 1)
subway2['무임승차비율'] = round((subway2['무임승차']  / subway2['전체탑승객']) * 100, 1)
subway2.head()

,사용월,호선명,역ID,지하철역,유임승차,유임하차,무임승차,무임하차,전체탑승객,유임승차비율,무임승차비율
0,2022-06,1호선,150,서울역,1150754,1123193,194717,187155,1345471,85.5,14.5
1,2022-06,1호선,151,시청,537067,543569,80671,78279,617738,86.9,13.1
2,2022-06,1호선,152,종각,834173,815704,131987,121717,966160,86.3,13.7
3,2022-06,1호선,153,종로3가,450317,404106,299617,278151,749934,60.0,40.0
4,2022-06,1호선,154,종로5가,394420,404105,254236,244555,648656,60.8,39.2


In [9]:
# 승차비율 50% 이하인 지하철역

---
## 출퇴근 시간대별 이용현황

In [8]:
import pandas as pd


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# %pip install xlrd
subwayDF = pd.read_excel('subway.xls',\
	sheet_name='지하철 시간대별 이용현황',\
	header=[0, 1], thousands=',')
subwayDF.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'subway.xls'

In [ ]:
commuteDF = subwayDF.iloc[:, [1, 3, 10, 12, 14]]
commuteDF.head(3)

,호선명,지하철역,07:00:00~07:59:59,08:00:00~08:59:59,09:00:00~09:59:59
,Unnamed: 1_level_1,Unnamed: 3_level_1,승차,승차,승차
0,1호선,서울역,39233,65106,54833
1,1호선,시청,6647,8294,9211
2,1호선,종각,5814,9612,11847


In [ ]:
commuteDF.dtypes

호선명                Unnamed: 1_level_1    object
지하철역               Unnamed: 3_level_1    object
07:00:00~07:59:59  승차                    object
08:00:00~08:59:59  승차                    object
09:00:00~09:59:59  승차                    object
dtype: object

In [ ]:
# instead thousands parameter
# for column in commuteDF.iloc[:, 2:]:
	# column.apply(lambda x: x.replace(',', ''))

In [ ]:
# 7~9시까지의 승차 데이터 타입 object -> int64로 변경
commuteDF.iloc[:, 2:].astype('int64')
commuteDF.iloc[:, 2:].dtypes

07:00:00~07:59:59  승차    int64
08:00:00~08:59:59  승차    int64
09:00:00~09:59:59  승차    int64
dtype: object

In [ ]:
commuteDF.iloc[:, :].head(3)

,호선명,지하철역,07:00:00~07:59:59,08:00:00~08:59:59,09:00:00~09:59:59
,Unnamed: 1_level_1,Unnamed: 3_level_1,승차,승차,승차
0,1호선,서울역,39233,65106,54833
1,1호선,시청,6647,8294,9211
2,1호선,종각,5814,9612,11847


In [ ]:
# 7~9시까지의 승차인원 데이터 추가
total = commuteDF.sum(axis=1, numeric_only=True)
commuteDF.insert(len(commuteDF.columns), '7~9시까지 승차인원', total)
commuteDF

,호선명,지하철역,07:00:00~07:59:59,08:00:00~08:59:59,09:00:00~09:59:59,7~9시까지 승차인원
,Unnamed: 1_level_1,Unnamed: 3_level_1,승차,승차,승차,
0,1호선,서울역,39233,65106,54833,159172
1,1호선,시청,6647,8294,9211,24152
2,1호선,종각,5814,9612,11847,27273
3,1호선,종로3가,4746,8589,13179,26514
4,1호선,종로5가,5020,8550,11925,25495
...,...,...,...,...,...,...
610,신림선,당곡,7708,11603,6862,26173
611,신림선,신림,3099,5402,2941,11442
612,신림선,서원,8427,12646,7590,28663
